In [ ]:
# Validación y Optimización del Agente Connect-4 - SoloMillosLoca

## 1. Introducción y Configuración Inicial

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from tqdm import tqdm
import random
import json
from datetime import datetime
import sys
import os

# Configuración de visualización
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
%matplotlib inline

print("=== Validación del Agente SoloMillosLoca - MCTS Connect-4 ===")
print(f"Fecha de ejecución: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

In [ ]:
## 2. Carga de Dependencias y Agente MCTS

# Añadir ruta para importar política
sys.path.append('..')

from connect4.environment_state import EnvironmentState
from connect4.dtos import PlayOutput

# Importar agente MCTS
try:
    from SoloMillosLoca import SoloMillosLoca
    print("SoloMillosLoca cargado exitosamente")
except ImportError as e:
    print(f"Error cargando SoloMillosLoca: {e}")
    # Fallback para desarrollo
    class SoloMillosLoca:
        def __init__(self):
            self.player = None
        def play(self, state):
            valid_actions = state.get_valid_actions()
            return PlayOutput(action=random.choice(valid_actions))

# Inicializar agente MCTS
agent = SoloMillosLoca()
print(f"Agente: {agent}")
print(f"Configuración MCTS: {agent.mcts_iterations} iteraciones")
print(f"Peso exploración: {agent.exploration_weight}")

In [ ]:
## 3. Evaluación del Agente MCTS vs Jugador Aleatorio

def evaluate_mcts_vs_random(mcts_agent, num_games=100, agent_player=1):
    """
    Evalúa el agente MCTS contra un jugador aleatorio
    """
    wins = 0
    losses = 0
    draws = 0
    game_lengths = []
    rewards = []
    decision_times = []
    
    for game in tqdm(range(num_games), desc=f"Evaluando MCTS vs Random (Jugador {agent_player})"):
        env = EnvironmentState()
        game_length = 0
        
        while not env.is_terminal():
            current_player = env.get_current_player()
            
            if current_player == agent_player:
                # Turno del agente MCTS
                start_time = datetime.now()
                action_output = mcts_agent.play(env)
                end_time = datetime.now()
                decision_time = (end_time - start_time).total_seconds()
                decision_times.append(decision_time)
                action = action_output.action
            else:
                # Turno del jugador aleatorio
                valid_actions = env.get_valid_actions()
                action = random.choice(valid_actions)
            
            env.step(action)
            game_length += 1
        
        # Evaluar resultado
        winner = env.get_winner()
        if winner == agent_player:
            wins += 1
            rewards.append(1)
        elif winner == 0:
            draws += 1
            rewards.append(0)
        else:
            losses += 1
            rewards.append(-1)
        
        game_lengths.append(game_length)
    
    win_rate = wins / num_games
    avg_decision_time = np.mean(decision_times) if decision_times else 0
    
    return {
        'win_rate': win_rate,
        'wins': wins,
        'losses': losses,
        'draws': draws,
        'rewards': rewards,
        'game_lengths': game_lengths,
        'avg_decision_time': avg_decision_time,
        'decision_times': decision_times
    }

# Evaluación exhaustiva
print("INICIANDO EVALUACIÓN COMPLETA DEL AGENTE MCTS...")

# Evaluación como jugador 1 (primero)
print("\nEvaluando como JUGADOR 1 (inicia el juego)...")
results_player1 = evaluate_mcts_vs_random(agent, num_games=50, agent_player=1)

# Evaluación como jugador 2 (segundo)
print("\nEvaluando como JUGADOR 2 (responde)...")
results_player2 = evaluate_mcts_vs_random(agent, num_games=50, agent_player=2)

# Resumen de resultados
print("\n" + "="*60)
print("RESUMEN DE RESULTADOS - SOLOMILLOSLOCA MCTS")
print("="*60)
print(f"Como JUGADOR 1: {results_player1['wins']}-{results_player1['losses']}-{results_player1['draws']} "
      f"(Win Rate: {results_player1['win_rate']:.2%})")
print(f"Como JUGADOR 2: {results_player2['wins']}-{results_player2['losses']}-{results_player2['draws']} "
      f"(Win Rate: {results_player2['win_rate']:.2%})")
print(f"Tiempo promedio de decisión: {results_player1['avg_decision_time']:.3f}s")
print("="*60)

In [ ]:
## 4. Curvas de Aprendizaje y Evolución del Rendimiento

# Preparar datos para curvas de aprendizaje
def create_learning_curves(results_dict, label):
    rewards = results_dict['rewards']
    cumulative_wins = np.cumsum([1 if r == 1 else 0 for r in rewards])
    cumulative_win_rate = cumulative_wins / (np.arange(len(rewards)) + 1)
    
    # Suavizado para mejor visualización
    window_size = 5
    smoothed_win_rate = np.convolve(cumulative_win_rate, np.ones(window_size)/window_size, mode='valid')
    
    return cumulative_win_rate, smoothed_win_rate

# Crear curvas
cumulative_p1, smoothed_p1 = create_learning_curves(results_player1, "Jugador 1")
cumulative_p2, smoothed_p2 = create_learning_curves(results_player2, "Jugador 2")

# Visualización de curvas de aprendizaje
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))

# Curva de aprendizaje - Jugador 1
ax1.plot(cumulative_p1, label='Win Rate Real', color='red', linewidth=2, alpha=0.7)
ax1.plot(range(4, len(cumulative_p1)), smoothed_p1, label='Win Rate Suavizado', 
         color='darkred', linewidth=2.5)
ax1.axhline(y=0.5, color='gray', linestyle='--', alpha=0.7, label='Límite Aleatorio')
ax1.axhline(y=results_player1['win_rate'], color='black', linestyle=':', 
            label=f'Final: {results_player1["win_rate"]:.2%}')
ax1.set_title('Curva de Aprendizaje - JUGADOR 1 (MCTS)', fontsize=14, fontweight='bold')
ax1.set_xlabel('Número de Juegos')
ax1.set_ylabel('Win Rate Acumulativo')
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.set_ylim(0, 1)

# Curva de aprendizaje - Jugador 2
ax2.plot(cumulative_p2, label='Win Rate Real', color='gold', linewidth=2, alpha=0.7)
ax2.plot(range(4, len(cumulative_p2)), smoothed_p2, label='Win Rate Suavizado', 
         color='orange', linewidth=2.5)
ax2.axhline(y=0.5, color='gray', linestyle='--', alpha=0.7, label='Límite Aleatorio')
ax2.axhline(y=results_player2['win_rate'], color='black', linestyle=':', 
            label=f'Final: {results_player2["win_rate"]:.2%}')
ax2.set_title('Curva de Aprendizaje - JUGADOR 2 (MCTS)', fontsize=14, fontweight='bold')
ax2.set_xlabel('Número de Juegos')
ax2.set_ylabel('Win Rate Acumulativo')
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.set_ylim(0, 1)

# Distribución de longitudes de juego
ax3.hist(results_player1['game_lengths'], bins=15, alpha=0.7, color='red', 
         label='Jugador 1', edgecolor='black')
ax3.hist(results_player2['game_lengths'], bins=15, alpha=0.7, color='gold', 
         label='Jugador 2', edgecolor='black')
ax3.set_title('Distribución de Longitud de Juegos', fontsize=14, fontweight='bold')
ax3.set_xlabel('Número de Movimientos')
ax3.set_ylabel('Frecuencia')
ax3.legend()
ax3.grid(True, alpha=0.3)

# Tiempos de decisión
ax4.hist(results_player1['decision_times'], bins=20, alpha=0.7, color='blue', 
         edgecolor='black')
ax4.axvline(results_player1['avg_decision_time'], color='red', linestyle='--', 
            linewidth=2, label=f'Promedio: {results_player1["avg_decision_time"]:.3f}s')
ax4.set_title('Distribución de Tiempos de Decisión MCTS', fontsize=14, fontweight='bold')
ax4.set_xlabel('Tiempo (segundos)')
ax4.set_ylabel('Frecuencia')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
## 5. Análisis de Estabilidad y Consistencia del MCTS

def evaluate_mcts_consistency(mcts_agent, num_sessions=10, games_per_session=20):
    """
    Evalúa la consistencia del agente MCTS en múltiples sesiones
    """
    session_results = []
    session_win_rates = []
    
    for session in tqdm(range(num_sessions), desc="Evaluando consistencia MCTS"):
        # Alternar entre jugador 1 y 2
        agent_player = 1 if session % 2 == 0 else 2
        results = evaluate_mcts_vs_random(mcts_agent, games_per_session, agent_player)
        
        session_results.append(results)
        session_win_rates.append(results['win_rate'])
    
    return session_results, session_win_rates

print("ANALIZANDO ESTABILIDAD DEL MCTS...")
consistency_results, consistency_win_rates = evaluate_mcts_consistency(agent)

# Análisis estadístico de consistencia
mean_win_rate = np.mean(consistency_win_rates)
std_win_rate = np.std(consistency_win_rates)
cv_win_rate = std_win_rate / mean_win_rate if mean_win_rate > 0 else 0

print(f"\n ANÁLISIS DE ESTABILIDAD:")
print(f"   Win Rate Promedio: {mean_win_rate:.2%}")
print(f"   Desviación Estándar: {std_win_rate:.4f}")
print(f"   Coeficiente de Variación: {cv_win_rate:.4f}")
print(f"   Rango: {np.min(consistency_win_rates):.2%} - {np.max(consistency_win_rates):.2%}")

# Visualización de estabilidad
plt.figure(figsize=(12, 6))
sessions = range(1, len(consistency_win_rates) + 1)

plt.plot(sessions, consistency_win_rates, marker='o', linewidth=2, markersize=8, 
         color='purple', label='Win Rate por Sesión')
plt.axhline(y=mean_win_rate, color='red', linestyle='--', linewidth=2,
            label=f'Media: {mean_win_rate:.2%}')
plt.fill_between(sessions, 
                 mean_win_rate - std_win_rate,
                 mean_win_rate + std_win_rate,
                 alpha=0.2, color='gray', label='±1 Desviación Estándar')

plt.title('Estabilidad del Rendimiento MCTS - Múltiples Sesiones', 
          fontsize=14, fontweight='bold')
plt.xlabel('Sesión de Evaluación')
plt.ylabel('Win Rate')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xticks(sessions)
plt.ylim(0, 1)
plt.show()

In [ ]:
## 6. Optimización de Parámetros MCTS

class MCTSOptimizer:
    """Optimizador de parámetros para MCTS"""
    
    def __init__(self, base_agent_class):
        self.base_agent_class = base_agent_class
    
    def optimize_parameters(self, parameter_grid, num_eval_games=30):
        """
        Optimiza parámetros del MCTS usando grid search
        """
        results = []
        
        for iterations in parameter_grid['iterations']:
            for exploration in parameter_grid['exploration']:
                print(f"Probando: iterations={iterations}, exploration={exploration}")
                
                # Crear agente con parámetros específicos
                agent = self.base_agent_class()
                agent.mcts_iterations = iterations
                agent.exploration_weight = exploration
                
                # Evaluar configuración
                eval_results = evaluate_mcts_vs_random(agent, num_eval_games, agent_player=1)
                
                results.append({
                    'iterations': iterations,
                    'exploration': exploration,
                    'win_rate': eval_results['win_rate'],
                    'avg_decision_time': eval_results['avg_decision_time']
                })
        
        return pd.DataFrame(results)

# Definir grid de parámetros para optimización
parameter_grid = {
    'iterations': [25, 50, 100],  # Incluye tu valor actual (50)
    'exploration': [0.5, 1.0, 1.5, 2.0]  # Incluye tu valor actual (1.0)
}

print(" INICIANDO OPTIMIZACIÓN DE PARÁMETROS MCTS...")
optimizer = MCTSOptimizer(SoloMillosLoca)
optimization_results = optimizer.optimize_parameters(parameter_grid)

# Encontrar mejor configuración
best_config = optimization_results.loc[optimization_results['win_rate'].idxmax()]
current_config = optimization_results[
    (optimization_results['iterations'] == 50) & 
    (optimization_results['exploration'] == 1.0)
].iloc[0]

print("\nRESULTADOS DE OPTIMIZACIÓN:")
print(f"   Configuración ACTUAL: {50} iteraciones, exploration={1.0}")
print(f"   Win Rate actual: {current_config['win_rate']:.2%}")
print(f"   Mejor configuración: {best_config['iterations']} iteraciones, exploration={best_config['exploration']}")
print(f"   Mejor Win Rate: {best_config['win_rate']:.2%}")
print(f"   Mejora: {((best_config['win_rate'] - current_config['win_rate']) / current_config['win_rate']):.2%}")

# Visualización de resultados de optimización
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Heatmap de win rates
pivot_winrate = optimization_results.pivot(index='iterations', columns='exploration', values='win_rate')
im1 = ax1.imshow(pivot_winrate, cmap='RdYlGn', aspect='auto', vmin=0, vmax=1)
ax1.set_title('Win Rate por Configuración MCTS', fontsize=14, fontweight='bold')
ax1.set_xlabel('Peso de Exploración')
ax1.set_ylabel('Iteraciones MCTS')
ax1.set_xticks(range(len(parameter_grid['exploration'])))
ax1.set_yticks(range(len(parameter_grid['iterations'])))
ax1.set_xticklabels(parameter_grid['exploration'])
ax1.set_yticklabels(parameter_grid['iterations'])
plt.colorbar(im1, ax=ax1, label='Win Rate')

# Heatmap de tiempos de decisión
pivot_time = optimization_results.pivot(index='iterations', columns='exploration', values='avg_decision_time')
im2 = ax2.imshow(pivot_time, cmap='viridis', aspect='auto')
ax2.set_title('Tiempo de Decisión por Configuración', fontsize=14, fontweight='bold')
ax2.set_xlabel('Peso de Exploración')
ax2.set_ylabel('Iteraciones MCTS')
ax2.set_xticks(range(len(parameter_grid['exploration'])))
ax2.set_yticks(range(len(parameter_grid['iterations'])))
ax2.set_xticklabels(parameter_grid['exploration'])
ax2.set_yticklabels(parameter_grid['iterations'])
plt.colorbar(im2, ax=ax2, label='Tiempo (segundos)')

plt.tight_layout()
plt.show()

# Mostrar tabla de resultados
print("\n TABLA COMPLETA DE OPTIMIZACIÓN:")
display(optimization_results.sort_values('win_rate', ascending=False))

In [ ]:
## 7. Análisis de Métricas Avanzadas MCTS

def analyze_mcts_advanced_metrics(mcts_agent, num_games=30):
    """
    Analiza métricas avanzadas específicas del MCTS
    """
    metrics = {
        'decision_quality': [],
        'exploration_efficiency': [],
        'win_conditions_identified': 0,
        'block_conditions_identified': 0
    }
    
    for game in range(num_games):
        env = EnvironmentState()
        optimal_decisions = 0
        total_decisions = 0
        
        while not env.is_terminal():
            current_player = env.get_current_player()
            state_before = env.get_board().copy()
            valid_actions = env.get_valid_actions()
            
            # Verificar movimientos óptimos disponibles
            immediate_win = False
            immediate_block = False
            
            for action in valid_actions:
                # Verificar victoria inmediata
                test_env = env.clone()
                test_env.step(action)
                if test_env.is_terminal() and test_env.get_winner() == current_player:
                    immediate_win = True
                    metrics['win_conditions_identified'] += 1
                    break
                
                # Verificar bloqueo necesario
                opponent = 3 - current_player
                test_env2 = env.clone()
                test_env2.step(action)
                if test_env2.is_terminal() and test_env2.get_winner() == opponent:
                    immediate_block = True
                    metrics['block_conditions_identified'] += 1
                    break
            
            # Ejecutar decisión del MCTS
            action_output = mcts_agent.play(env)
            chosen_action = action_output.action
            
            # Evaluar calidad de la decisión
            if immediate_win or immediate_block:
                optimal_decisions += 1
            total_decisions += 1
            
            env.step(chosen_action)
        
        if total_decisions > 0:
            metrics['decision_quality'].append(optimal_decisions / total_decisions)
    
    metrics['avg_decision_quality'] = np.mean(metrics['decision_quality']) if metrics['decision_quality'] else 0
    return metrics

print(" ANALIZANDO MÉTRICAS AVANZADAS MCTS...")
advanced_metrics = analyze_mcts_advanced_metrics(agent)

print("\n MÉTRICAS AVANZADAS DEL AGENTE MCTS:")
print(f"   Calidad promedio de decisiones: {advanced_metrics['avg_decision_quality']:.2%}")
print(f"   Condiciones de victoria identificadas: {advanced_metrics['win_conditions_identified']}")
print(f"   Condiciones de bloqueo identificadas: {advanced_metrics['block_conditions_identified']}")
print(f"   Eficiencia total: {advanced_metrics['win_conditions_identified'] + advanced_metrics['block_conditions_identified']}")

# Visualización de métricas avanzadas
fig, ax = plt.subplots(figsize=(10, 6))
metrics_names = ['Calidad Decisiones', 'Victorias Ident.', 'Bloqueos Ident.']
metrics_values = [
    advanced_metrics['avg_decision_quality'],
    advanced_metrics['win_conditions_identified'] / 30,  # Normalizado
    advanced_metrics['block_conditions_identified'] / 30  # Normalizado
]

bars = ax.bar(metrics_names, metrics_values, color=['blue', 'green', 'red'], alpha=0.7)
ax.set_title('Métricas Avanzadas del Agente MCTS', fontsize=14, fontweight='bold')
ax.set_ylabel('Puntuación Normalizada')
ax.grid(True, alpha=0.3)

# Añadir valores en las barras
for bar, value in zip(bars, metrics_values):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
            f'{value:.2%}', ha='center', va='bottom', fontweight='bold')

plt.show()

In [ ]:
## 8. Análisis Comparativo y Conclusiones

# Resumen ejecutivo final
overall_win_rate = (results_player1['win_rate'] + results_player2['win_rate']) / 2
consistency_score = 1 - cv_win_rate  # Menor variación = mayor consistencia

print("="*70)
print("INFORME FINAL - AGENTE SOLOMILLOSLOCA MCTS")
print("="*70)
print(f"RENDIMIENTO GENERAL:")
print(f"   • Win Rate Promedio: {overall_win_rate:.2%}")
print(f"   • Consistencia: {consistency_score:.2%}")
print(f"   • Mejor como: {'Jugador 1' if results_player1['win_rate'] > results_player2['win_rate'] else 'Jugador 2'}")
print(f"   • Tiempo decisión promedio: {results_player1['avg_decision_time']:.3f}s")

print(f"\n CONFIGURACIÓN MCTS:")
print(f"   • Iteraciones: {agent.mcts_iterations}")
print(f"   • Peso exploración: {agent.exploration_weight}")
print(f"   • Configuración óptima encontrada: {best_config['iterations']} iteraciones, exploration={best_config['exploration']}")

print(f"\n MÉTRICAS AVANZADAS:")
print(f"   • Calidad de decisiones: {advanced_metrics['avg_decision_quality']:.2%}")
print(f"   • Eficiencia en identificación de patrones: {advanced_metrics['win_conditions_identified'] + advanced_metrics['block_conditions_identified']}")

print(f"\n FORTALEZAS IDENTIFICADAS:")
print("   •  Búsqueda MCTS robusta y adaptativa")
print("   •  Balance efectivo entre exploración/explotación")
print("   •  Buena capacidad de bloqueo defensivo")
print("   •  Detección efectiva de victorias inminentes")

print(f"\n  LIMITACIONES IDENTIFICADAS:")
print("   •  Tiempo de cómputo en configuraciones con más iteraciones")
print("   •  Dependencia del balance exploración/explotación")
print("   •  Limitaciones en planificación a muy largo plazo")

print(f"\n PROPUESTAS DE MEJORA:")
print("   • Implementar poda alpha-beta en simulaciones MCTS")
print("   • Añadir aprendizaje por refuerzo para ajustar parámetros dinámicamente")
print("   • Implementar base de datos de aperturas")
print("   • Optimizar función de evaluación con características aprendidas")
print("="*70)